# Keras auto-compressor

This notebook was designed as an example to test the `keras_autocompressor` 
python module with a custom MobileNetV2 architecture. 

Here we change the model optimizer as the `tf.keras.optimizers.SGD` instead of 
`tf.keras.optimizers.Adam`.

In [ ]:
# Import basic modules
import tensorflow as tf
import keras_tuner as kt
import tensorflow_datasets as tfds

In [ ]:
# Import the module functions to create the our custom model definition
from keras_autocompressor.metrics import AccuracyCompression
from keras_autocompressor.hypermodels import HyperCompressedMobileNetV2

# Define the custom model definition
class CustomHyperMobileNetV2(HyperCompressedMobileNetV2):
    def __init__(
            self, 
            max_parameters: int, 
            num_classes: 
            int, tau: float = 0.8, 
            name=None, tunable=True
        ):
        super().__init__(max_parameters, num_classes, tau, name, tunable)

    # Custom model definition
    def build(self, hp: kt.HyperParameters):
       
        # Select the back-bone for the model
        backbone = self.create_backbone(hp)

        # Freeze the base model
        backbone.trainable = False

        # Create the new top for the network
        x = backbone.output
        x = tf.keras.layers.GlobalAveragePooling2D(
            name='top_gap')(x)
        x = tf.keras.layers.Dense(
            hp.Int('top_fc1_units', 4, 32, step=8, default=4), name='top_fc1')(x)
        x = tf.keras.layers.Dense(
            self._num_classes, name='classifier', activation='softmax')(x)

        # Create the new model
        model = tf.keras.Model(
            inputs=backbone.inputs, 
            outputs=x, 
            name='autosearch'
        )

        # Calculate the compression rate for the proposed metric
        actual_params = model.count_params()
        params_rate = actual_params / self._max_parameters
        compression_rate = 1 - params_rate

        # Create the custom metric for this model
        accuracy_compression_metric = AccuracyCompression(
            name='acc_comp', 
            compression_rate=compression_rate, 
            tau=self._tau
        )
        
        # Compile the model
        model.compile(
            optimizer=tf.keras.optimizers.SGD(
                hp.Float('lr', 1E-5, 1E-2, sampling='log')), 
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=['accuracy', accuracy_compression_metric]
        )

        return model

In [ ]:
# Experiment settings
tensorflow_dataset = 'horses_or_humans'
batch_size = 16
max_search_epochs = 5

In [ ]:
# Load a dataset for training and testing
(ds_train, ds_test), ds_info = tfds.load(
    tensorflow_dataset, 
    split=['train', 'test'], 
    shuffle_files=True, 
    with_info=True, 
    as_supervised=True
)

In [ ]:
# Plot some dataset samples
fig = tfds.show_examples(ds_train, ds_info)

In [ ]:
# Generate pre-processing function for the images
def preprocess_images(
        image, 
        label, 
        num_classes=ds_info.features['label'].num_classes
    ):
    # Resize images
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224,224))

    # Preprocess with the MobileNet function
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)

    # Change labels to categorical
    label = tf.cast(
        tf.one_hot(tf.cast(label, tf.int32), num_classes), dtype=tf.float32
    )
    
    return image, label

In [ ]:
# Pre-process the images
ds_train = ds_train.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.map(preprocess_images, num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
# Create batches for inference in both subsets
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)   

ds_test = ds_test.batch(batch_size)

In [ ]:
# Create our custom Hyper model
hyper_model = CustomHyperMobileNetV2(
    max_parameters=2.3E6,
    num_classes=ds_info.features['label'].num_classes,
    tau=0.8,
)

# Create the tuner object for our search
mobilenetv2_compressor = kt.Hyperband(
    hyper_model,
    max_epochs=max_search_epochs,
    objective=kt.Objective("val_acc_comp", direction="max"),
    directory='./logs/custom_mobilenetv2/',
    project_name=tensorflow_dataset,
    overwrite=True
)

In [ ]:
# Run the hyperparameters search + auto compression
mobilenetv2_compressor.search(ds_train, validation_data=ds_test)

In [ ]:
# Get the best hyper parameters after the search
best_hyperparameters = mobilenetv2_compressor.get_best_hyperparameters()[0]
for key, item in best_hyperparameters.values.items():
    print(f'Hyperparameter: {key:20} | Value: {item}')

In [ ]:
# Get the best model
best_model = mobilenetv2_compressor.get_best_models()[0]

In [ ]:
# Compute the performance for the top-5 models obtained
for idx, sub_model in enumerate(mobilenetv2_compressor.get_best_models(5), start=1):
    metrics = sub_model.evaluate(ds_test, verbose=0)
    print(f'Top-{idx} model | val_accuracy: {metrics[1]:0.4f}  | params:' \
          + f' {sub_model.count_params()}')

In [ ]:
# Display the top-3 best models and their hyperparameters within the search
mobilenetv2_compressor.results_summary(num_trials=3)